In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

2022-11-27 20:19:14.249247: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

2022-11-27 20:19:23.768097: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 20:19:23.833414: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 20:19:23.834142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-27 20:19:23.836175: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp6oti7w_x/assets


INFO:tensorflow:Assets written to: /tmp/tmp6oti7w_x/assets
2022-11-27 20:19:27.531788: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-27 20:19:27.531828: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-27 20:19:27.532479: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp6oti7w_x
2022-11-27 20:19:27.533884: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-11-27 20:19:27.533912: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmp6oti7w_x
2022-11-27 20:19:27.539006: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-27 20:19:27.542526: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-11-27 20:19:27.701296: I tensorflow/cc/saved_model/loader.cc:212] Running initializatio

In [5]:
with open('tfl_model', 'wb') as f_out:
    f_out.write(tflite_model)

### Q1 What's the size of the converted model?

In [6]:
from pathlib import Path
Path('tfl_model').stat()
size = Path('tfl_model').stat().st_size
round((size*0.000001), 1)

44.9

In [7]:
import tensorflow.lite as tflite

In [8]:
interpriter = tflite.Interpreter(model_path='tfl_model')
interpriter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [9]:
input_index = interpriter.get_input_details()[0]['index']

### Q2 What's the output index for this model?

In [10]:
output_index = interpriter.get_output_details()[0]['index']
output_index

13

In [11]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [12]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [13]:
img = download_image('https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg')

In [14]:
img_prep = prepare_image(img, (150,150))
img_prep.save('test_img.jpg')

/tmp/ipykernel_57/2487457277.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


### Q3 What's the value in the first pixel, the R channel?

In [15]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
train_gen = ImageDataGenerator(rescale=1./255)

In [18]:
test_image = train_gen.flow_from_directory(
    './Pred',
    target_size=(150,150)
)

Found 1 images belonging to 2 classes.


In [19]:
test_image[0][0][0][0][0][0]

0.54901963

### Q4 What's the output of the model?

In [23]:
img = load_img('test_img.jpg', target_size=(150, 150))

x = np.array(img)
X = np.array([x])

In [25]:
preds = model.predict(test_image)
preds

1/1 [==============================] - 0s 80ms/step


array([[0.81696784]], dtype=float32)

### Q5 What's the size of this base image?
% docker images  
REPOSITORY  
TAG                IMAGE ID       CREATED       SIZE  
svizor42/zoomcamp-dino-dragon-lambda   v2 20ef58b21a05   9 days ago    639MB